In [57]:
# imports
import pandas as pd


In [58]:
lst = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']
haps = ['MATERNAL','PATERNAL']
chrNames = []
for i in range(len(lst)):
    for j in range(len(haps)):
        chrNames.append(f'chr{lst[i]}_{haps[j]}')
chrNames.remove('chrX_PATERNAL')
chrNames.remove('chrY_MATERNAL')
print(len(chrNames))

46


# Filtering annotations for just S1, S2, S3 for HG002

In [59]:
chrNames = ['chr1_MATERNAL', 'chr1_PATERNAL', 'chr2_MATERNAL', 'chr2_PATERNAL', 'chr3_MATERNAL', 'chr3_PATERNAL', 'chr4_MATERNAL', 'chr4_PATERNAL', 'chr5_MATERNAL', 'chr5_PATERNAL', 'chr6_MATERNAL', 'chr6_PATERNAL', 'chr7_MATERNAL', 'chr7_PATERNAL', 'chr8_MATERNAL', 'chr8_PATERNAL', 'chr9_MATERNAL', 'chr9_PATERNAL', 
            'chr10_MATERNAL', 'chr10_PATERNAL', 'chr11_MATERNAL', 'chr11_PATERNAL', 'chr12_MATERNAL', 'chr12_PATERNAL', 'chr13_MATERNAL', 'chr13_PATERNAL', 'chr14_MATERNAL_v1', 'chr14_PATERNAL', 'chr15_MATERNAL', 'chr15_PATERNAL', 'chr16_MATERNAL', 'chr16_PATERNAL', 'chr17', 'chr17_PATERNAL', 'chr18_MATERNAL', 
            'chr18_PATERNAL', 'chr19_MATERNAL', 'chr19_PATERNAL', 'chr20_MATERNAL', 'chr20_PATERNAL', 'chr21_MATERNAL', 'chr21_PATERNAL', 'chr22_MATERNAL', 'chr22_PATERNAL', 'chrX_MATERNAL', 'chrY_PATERNAL']
for i in chrNames:
    input_bed = f'/Users/emilyxu/Desktop/cdrproject/AS-HOR+SF-vs-HG002/AS-HOR+SF-vs-HG002_asm_v0.6_{i}.bed'
    output_bed = f'/Users/emilyxu/Desktop/cdrproject/filteredHG002Annotations/HG002-humanSC{i}.bed'
    with open(input_bed, 'r') as in_file, open(output_bed, 'w') as out_file:
        for line in in_file:
            fields = line.rstrip().split()
            if fields[3].startswith(('S1','S2','S3','S4','S01','S02','S03')):
                out_file.write(line)


# Storing all unique CDRs from bedfiles as pandas dataframe 

### Input: Directory of CDR BED files for each chromosome

In [60]:
import glob
path = '/Users/emilyxu/Desktop/cdrproject/filteredIntersectsOutputs'
filenames = glob.glob(path + "/*.bed")
print('num of files: ', len(filenames))

# store all files into one dataframe
filteredBEDfiles = pd.DataFrame()
for file in filenames:
    df = pd.read_csv(file, sep='\t', header=None)
    header = ['chr', 'start', 'end']
    df.columns = header[:len(df.columns)]
    filteredBEDfiles = pd.concat([filteredBEDfiles, df])

filteredBEDfiles['range'] = list(zip(filteredBEDfiles['start'], filteredBEDfiles['end']))
filteredUniques = filteredBEDfiles.groupby('range').first().reset_index()

filteredUniques['length'] = filteredUniques['end'] - filteredUniques['start']
print('Statistics Before filtering for length:')
print(filteredUniques.describe())
filteredUniques = filteredUniques[filteredUniques['length'] > 6000]
print()
print('Statistics After filtering for length (>6000bp):')
print(filteredUniques.describe())
print('All Unique CDRs')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # print entire dataframe out
    print(filteredUniques.sort_values('length'))

num of files:  46
Statistics Before filtering for length:
              start           end        length
count  4.870000e+02  4.870000e+02    487.000000
mean   3.053413e+07  3.054481e+07  10673.511294
std    2.900074e+07  2.900340e+07  10664.070700
min    1.487334e+06  1.490334e+06   3000.000000
25%    7.340538e+06  7.344580e+06   4000.000000
50%    2.572666e+07  2.573066e+07   7000.000000
75%    4.717884e+07  4.718384e+07  13000.000000
max    1.246326e+08  1.246376e+08  83000.000000

Statistics After filtering for length (>6000bp):
              start           end        length
count  2.450000e+02  2.450000e+02    245.000000
mean   3.988381e+07  3.990114e+07  17334.693878
std    3.083722e+07  3.083793e+07  11653.212303
min    2.120334e+06  2.131334e+06   7000.000000
25%    9.531563e+06  9.555563e+06  10000.000000
50%    3.581664e+07  3.583664e+07  13000.000000
75%    5.582629e+07  5.583829e+07  22000.000000
max    1.245836e+08  1.246026e+08  83000.000000
All Unique CDRs
            

# Store unique CDRs as bedfiles for each chromosome

In [61]:
def createFile(df):
    """
    Function for creating new BED files that store just the 
    unique CDRs for each chromosome
    """
    chrName = df['chr'].iloc[0]
    df = df[['chr', 'start', 'end']]
    df.to_csv('/Users/emilyxu/Desktop/cdrproject/filteredIntersectUniques/' + chrName + '_filtered.bed',sep='\t', header=False, index=False)
for i in range(len(chrNames)):
    newDF = filteredUniques[(filteredUniques['chr'] == chrNames[i])]
    newDF = newDF[['chr', 'start', 'end']]
    createFile(newDF)

# Getting statistics for each chromosome's CDRs to look at distribution of CDR sizes

In [62]:
filteredUniques['length'] = filteredUniques['end'] - filteredUniques['start']
filteredUniques

for i in chrNames:
    chr = filteredUniques[filteredUniques['chr']==i]
    print(i)
    print(chr.describe())
    print()

chr1_MATERNAL
              start           end        length
count  5.000000e+00  5.000000e+00      5.000000
mean   1.226954e+08  1.227152e+08  19800.000000
std    1.056267e+05  9.545523e+04  13608.820669
min    1.225874e+08  1.226094e+08   8000.000000
25%    1.226104e+08  1.226524e+08  10000.000000
50%    1.226704e+08  1.226874e+08  17000.000000
75%    1.227784e+08  1.227884e+08  22000.000000
max    1.228304e+08  1.228384e+08  42000.000000

chr1_PATERNAL
              start           end        length
count  6.000000e+00  6.000000e+00      6.000000
mean   1.245148e+08  1.245336e+08  18833.333333
std    6.355287e+04  6.064322e+04   3868.677638
min    1.244216e+08  1.244446e+08  12000.000000
25%    1.244746e+08  1.244959e+08  18250.000000
50%    1.245231e+08  1.245416e+08  19000.000000
75%    1.245649e+08  1.245784e+08  21250.000000
max    1.245836e+08  1.246026e+08  23000.000000

chr2_MATERNAL
              start           end        length
count  5.000000e+00  5.000000e+00      5.000

# Looking at HG002 closest outputs

In [63]:
import glob
path = '/Users/emilyxu/Desktop/cdrproject/filteredClosest'
filenames = glob.glob(path + "/*.bed")
print('File names: ', filenames)
print('num of files: ', len(filenames))

# store all file info into one dataframe
closestBED = pd.DataFrame()
for file in filenames:
    df = pd.read_csv(file, sep='\t', header=None)
    header = ['chr1', 'start1', 'end1', 'chr2', 'start2', 'end2', 'distance']
    # df['haplotype'] = str(file.split('/')[-1]).split('_')[1]
    df.columns = header[:len(df.columns)]
    closestBED = pd.concat([closestBED, df])

closestBED = closestBED.loc[closestBED['distance'] != 0]
for i in chrNames:
    chr = closestBED[closestBED['chr1']==i]

print(closestBED.describe())


File names:  ['/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr3_PATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr15_PATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr21_PATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr12_MATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr4_MATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chrX_MATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr21_MATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr4_PATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr12_PATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr15_MATERNAL_closest_filtered.bed', '/Users/emilyxu/Desktop/cdrproject/filteredClosest/chr3_MATERNAL_closest_filtered.bed', '/Users/emil

# Reading in the clusters as a pd dataframe

In [64]:
path1='/Users/emilyxu/Desktop/cdrproject/filteredClusters/'

clusters = pd.DataFrame()
for i in chrNames:
    in_fh1=open(f'{path1}{i}_cluster_filtered.bed','r')
    df = pd.read_csv(in_fh1, sep='\t', header=None)
    header = ['chromosome', 'start', 'end', 'cluster']
    df.columns = header[:len(df.columns)]
    cluster = df[['chromosome','start', 'end']].groupby(df['cluster'])
    result = cluster.agg({'chromosome':['min'],'start': ['count', 'min'],'end': ['count', 'max']})
    clusters = pd.concat([clusters, result])

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(clusters)


                chromosome start              end           
                       min count        min count        max
cluster                                                     
1            chr1_MATERNAL     5  122587375     5  122838375
1            chr1_PATERNAL     6  124421626     6  124602626
1            chr2_MATERNAL     5   92772650     5   93022650
1            chr2_PATERNAL     5   93149091     5   93305091
1            chr3_MATERNAL     5   92518177     5   92722177
1            chr3_PATERNAL     4   91881756     4   92042756
1            chr4_MATERNAL     7   51513881     7   51695881
1            chr4_PATERNAL     6   51351025     6   51522025
1            chr5_MATERNAL     2   49171085     2   49243085
2            chr5_MATERNAL     5   49440085     5   49550085
1            chr5_PATERNAL     8   55826289     8   56027289
1            chr6_MATERNAL     4   63671121     4   63778121
1            chr6_PATERNAL     4   63155740     4   63269740
1            chr7_MATERN

# Store clustered CDRs as bedfiles for each chromosome

In [65]:
for i in chrNames:
    single = clusters[clusters[('chromosome','min')] == i]
    df = pd.DataFrame(single, columns=[('chromosome','min'),('start','min'),('end','max')])
    print(df)
    df.to_csv(f'filteredClusterBEDs/{i}_cluster.bed', index=False, header=False, sep='\t')


            chromosome      start        end
                   min        min        max
cluster                                     
1        chr1_MATERNAL  122587375  122838375
            chromosome      start        end
                   min        min        max
cluster                                     
1        chr1_PATERNAL  124421626  124602626
            chromosome     start       end
                   min       min       max
cluster                                   
1        chr2_MATERNAL  92772650  93022650
            chromosome     start       end
                   min       min       max
cluster                                   
1        chr2_PATERNAL  93149091  93305091
            chromosome     start       end
                   min       min       max
cluster                                   
1        chr3_MATERNAL  92518177  92722177
            chromosome     start       end
                   min       min       max
cluster                               